Set Shots - Expected Goal Model - Model Tuning and Building - GBM

In [61]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import set_shots_file_path
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [63]:
model_version = 7
model_name = 'expected_goal_set'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [64]:
RESPONSE = ModellingDataContract.RESPONSE_GOAL

In [66]:
FEATURES = ModellingDataContract.set_goal_modelling_feature_list

In [83]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_set_goal

Load Data

In [84]:
df_modelling = pd.read_csv(set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet
11052,208,possGain,behind,1663.0,4,634,683.0,Geelong,Geelong,Tom Hawkins,Tom_Hawkins,Kick,41.0,25.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,1,0,105.0,32.0,73.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,48.0,48.0,26.0,25.0,21.0,21.0,14.0,683.0,644.0,642.0,641.0,49.0,39.0,8.062258,39.0,25.0,46.324939,0.570040,32.660913,0.116867,6.695990,0.364585,20.889203,1521.0,3.663562,True,True,False,False,True,False,False,True,False,False
11053,212,kickIn,goal,1730.0,4,787,885.0,Geelong,Geelong,Patrick Dangerfield,Patrick_Dangerfield,Kick,51.0,43.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,107.0,33.0,74.0,Kick,Contested Mark,Contest Target,Kick Inside 50 Result,51.0,53.0,53.0,53.0,43.0,38.0,38.0,38.0,885.0,852.0,851.0,850.0,98.0,33.0,5.385165,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,False,False,True,True,False,False,True,False,False
11054,213,centreBounce,goal,1754.0,4,956,979.0,Sydney,Sydney,Paddy McCartin,Paddy_McCartin,Kick,66.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,119.0,33.0,86.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,66.0,74.0,74.0,27.0,0.0,0.0,0.0,17.0,979.0,977.0,968.0,967.0,23.0,2.0,8.000000,14.0,0.0,14.000000,0.000000,0.000000,0.482343,27.636207,2.588598,148.315731,196.0,2.639057,True,True,False,False,True,False,False,False,True,False
11055,219,centreBounce,goal,1846.0,4,1252,1266.0,Sydney,Sydney,Chad Warner,Chad_Warner,Kick,26.0,3.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,125.0,39.0,86.0,Kick,Uncontested Mark,Ground Kick,Handball,26.0,18.0,-5.0,-2.0,3.0,2.0,-4.0,3.0,1266.0,1265.0,1264.0,1260.0,14.0,1.0,8.062258,54.0,3.0,54.083269,0.055499,3.179830,0.118569,6.793500,0.365993,20.969862,2916.0,3.988984,True,True,False,False,True,False,False,True,False,False
11056,228,centreBounce,goal,1912.0,4,1519,1560.0,Geelong,Geelong,Jeremy Cameron,Jeremy_Cameron,Kick,41.0,21.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,131.0,45.0,86.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,49.0,49.0,25.0,21.0,17.0,17.0,33.0,1560.0,1535.0,1532.0,1531.0,41.0,25.0,8.944272,39.0,21.0,44.294469,0.493941,28.300756,0.127885,7.327246,0.400462,22.944787,1521.0,3.663562,True,True,False,False,True,False,False,True,False,False


In [85]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [86]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [87]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [88]:
y_train.mean(), y_test.mean(), y_cal.mean()

(0.5412662521198417, 0.5415913200723327, 0.5415488976823064)

In [89]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
7,76,possGain,goal,723.0,2,604,637.0,Sydney,Sydney,Errol Gulden,Errol_Gulden,Kick,38.0,5.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,0,6,20.0,34.0,-14.0,Kick,Uncontested Mark,Kick,Uncontested Mark,38.0,45.0,31.0,35.0,5.0,4.0,8.0,8.0,637.0,609.0,608.0,608.0,33.0,28.0,7.071068,40.0,5.0,40.311289,0.124355,7.125016,0.158537,9.083529,0.501031,28.706948,1600.0,3.688879,True,False,True,False,True,False,False,True,False,False,0,0,0,1,0
8,84,centreBounce,goal,799.0,2,919,983.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,42.0,-2.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,6,0,26.0,46.0,-20.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,42.0,50.0,50.0,25.0,-2.0,-2.0,-2.0,-27.0,983.0,950.0,944.0,943.0,64.0,33.0,8.000000,36.0,2.0,36.055513,0.055499,3.179830,0.178638,10.235197,0.572261,32.788153,1296.0,3.583519,True,False,True,False,True,False,False,True,False,False,0,1,0,0,0
16,158,possGain,goal,1404.0,3,1430,1472.0,Sydney,Sydney,Sam Reid,Sam_Reid,Kick,61.0,34.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,0,6,58.0,83.0,-25.0,Kick,Contested Mark,Contest Target,Kick Inside 50 Result,61.0,64.0,64.0,64.0,34.0,31.0,31.0,31.0,1472.0,1446.0,1445.0,1443.0,42.0,26.0,4.242641,17.0,34.0,38.013156,1.107149,63.434949,0.075831,4.344825,0.241270,13.823765,289.0,2.833213,True,False,True,False,False,False,True,False,True,False,0,0,0,1,0
17,168,centreBounce,miss,1480.0,3,1850,1899.0,Brisbane Lions,Brisbane Lions,Eric Hipwood,Eric_Hipwood,Kick,52.0,-31.0,clanger,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,1,0,0,0,65.0,101.0,-36.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,52.0,57.0,57.0,10.0,-31.0,-26.0,-25.0,-5.0,1899.0,1860.0,1859.0,1858.0,49.0,39.0,7.071068,26.0,31.0,40.459857,0.872894,50.013114,0.102289,5.860740,0.323140,18.514573,676.0,3.258097,True,False,True,False,True,False,False,True,False,False,0,1,0,0,0
23,224,possGain,goal,1890.0,4,1515,1529.0,Brisbane Lions,Brisbane Lions,Eric Hipwood,Eric_Hipwood,Kick,41.0,46.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,6,0,78.0,122.0,-44.0,Kick,Uncontested Mark,Kick,Handball Received,41.0,45.0,19.0,18.0,46.0,44.0,57.0,57.0,1529.0,1523.0,1521.0,1519.0,14.0,6.0,4.472136,37.0,46.0,59.033889,0.893410,51.188616,0.068149,3.904627,0.209382,11.996711,1369.0,3.610918,True,False,True,False,False,False,True,True,False,False,0,0,0,1,0


Preprocess Data

In [90]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [91]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [92]:
X_train_preproc.shape, X_test_preproc.shape

((7076, 9), (2212, 9))

In [93]:
X_train_preproc.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn
4,33.0,46.572524,0.260602,0.133410,0,1,0,0,0
5,52.0,44.407207,0.945311,0.084822,0,0,0,1,0
6,47.0,46.754679,0.845930,0.091187,0,0,0,1,0
9,59.0,19.924859,0.305879,0.314407,0,0,0,1,0
10,40.0,52.345009,0.758378,0.089092,0,0,0,1,0


In [94]:
X_test_preproc.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn
7,38.0,40.311289,0.124355,0.158537,0,0,0,1,0
8,42.0,36.055513,0.055499,0.178638,0,1,0,0,0
16,61.0,38.013156,1.107149,0.075831,0,0,0,1,0
17,52.0,40.459857,0.872894,0.102289,0,1,0,0,0
23,41.0,59.033889,0.893410,0.068149,0,0,0,1,0


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [95]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [96]:
xgb_tuner.training_data.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn
4,33.0,46.572524,0.260602,0.133410,0,1,0,0,0
5,52.0,44.407207,0.945311,0.084822,0,0,0,1,0
6,47.0,46.754679,0.845930,0.091187,0,0,0,1,0
9,59.0,19.924859,0.305879,0.314407,0,0,0,1,0
10,40.0,52.345009,0.758378,0.089092,0,0,0,1,0


In [97]:
xgb_tuner.tune_hyperparameters()

[I 2023-07-11 14:02:13,874] A new study created in memory with name: no-name-b700229b-f240-41a6-8153-ba85b56b682c
[I 2023-07-11 14:02:13,911] Trial 0 finished with value: 0.6072736136616708 and parameters: {'max_depth': 5, 'min_child_weight': 66, 'eta': 0.43724996259441606, 'gamma': 0.8708327840706026, 'lambda': 5.7267449838869915, 'alpha': 0.000900195618396828, 'subsample': 0.39146852299846224, 'colsample_bytree': 0.648999376496223}. Best is trial 0 with value: 0.6072736136616708.
[I 2023-07-11 14:02:13,927] Trial 1 finished with value: 0.604604782551209 and parameters: {'max_depth': 2, 'min_child_weight': 44, 'eta': 0.9049377426170772, 'gamma': 0.9001907763746201, 'lambda': 0.013352171963000072, 'alpha': 0.023020735999908906, 'subsample': 0.603322566500178, 'colsample_bytree': 0.2544384213562794}. Best is trial 1 with value: 0.604604782551209.
[I 2023-07-11 14:02:13,945] Trial 2 finished with value: 0.6331056686128187 and parameters: {'max_depth': 12, 'min_child_weight': 58, 'eta': 0

Number of finished trials:  1000
Best trial:
  Value: 0.5717185752506849
  Params: 
    max_depth: 18
    min_child_weight: 35
    eta: 0.4696886428548536
    gamma: 0.0472567436880216
    lambda: 0.007222959316665699
    alpha: 0.005243627004085313
    subsample: 0.6820665382152401
    colsample_bytree: 0.8180404302533698


In [98]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 18,
 'min_child_weight': 35,
 'eta': 0.4696886428548536,
 'gamma': 0.0472567436880216,
 'lambda': 0.007222959316665699,
 'alpha': 0.005243627004085313,
 'subsample': 0.6820665382152401,
 'colsample_bytree': 0.8180404302533698}

Training Model - SuperXGBClassifier class for training and predictions

In [99]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [100]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [101]:
super_xgb.fit()

[0]	validation_0-logloss:0.63412	validation_1-logloss:0.63792
[1]	validation_0-logloss:0.61041	validation_1-logloss:0.61395
[2]	validation_0-logloss:0.60086	validation_1-logloss:0.60403
[3]	validation_0-logloss:0.59611	validation_1-logloss:0.60051
[4]	validation_0-logloss:0.59384	validation_1-logloss:0.59899
[5]	validation_0-logloss:0.59244	validation_1-logloss:0.59827
[6]	validation_0-logloss:0.59215	validation_1-logloss:0.59828
[7]	validation_0-logloss:0.59187	validation_1-logloss:0.59861
[8]	validation_0-logloss:0.59182	validation_1-logloss:0.59864
[9]	validation_0-logloss:0.59184	validation_1-logloss:0.59842
[10]	validation_0-logloss:0.59172	validation_1-logloss:0.59904
[11]	validation_0-logloss:0.59175	validation_1-logloss:0.59937
[12]	validation_0-logloss:0.59183	validation_1-logloss:0.59983
[13]	validation_0-logloss:0.59181	validation_1-logloss:0.59957
[14]	validation_0-logloss:0.59192	validation_1-logloss:0.60009
[15]	validation_0-logloss:0.59189	validation_1-logloss:0.60107
[1

In [102]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8180404302533698, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0472567436880216, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.4696886428548536, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=18, max_leaves=None,
              min_child_weight=35, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': -1,
                                    'Distance_to_Middle_Goal': -1,
                                    'Visible_Goal_Angle': 1, 'x0': -1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [103]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle',
 'ballUp',
 'centreBounce',
 'kickIn',
 'possGain',
 'throwIn']

In [104]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [105]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [106]:
super_xgb.calibrate()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [107]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)
cal_cal_probas = super_xgb.predict_proba(X_cal_preproc, calibrate=True)

Check Average Predictions

In [108]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.5373624, 0.5412662521198417, 0.5368447441092646)

In [109]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.537491, 0.5415913200723327, 0.5369921714739258)

In [110]:
cal_probas.mean(), cal_data[RESPONSE].mean(), cal_cal_probas.mean()

(0.54351795, 0.5415488976823064, 0.5415489413998579)

Export model

In [111]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [112]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,59,578.0,1,33.0,46.572524,0.260602,0.133410,0,1,0,0,0,1,0.552535,0.525995
1,202101_BrisbaneLions_Sydney,62,621.0,1,52.0,44.407207,0.945311,0.084822,0,0,0,1,0,0,0.356250,0.389181
2,202101_BrisbaneLions_Sydney,72,696.0,1,47.0,46.754679,0.845930,0.091187,0,0,0,1,0,0,0.386986,0.408780
3,202101_BrisbaneLions_Sydney,101,929.0,1,59.0,19.924859,0.305879,0.314407,0,0,0,1,0,1,0.929429,0.917519
4,202101_BrisbaneLions_Sydney,108,982.0,1,40.0,52.345009,0.758378,0.089092,0,0,0,1,0,0,0.252313,0.325553


In [113]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,76,723.0,1,38.0,40.311289,0.124355,0.158537,0,0,0,1,0,1,0.694763,0.649727
1,202101_BrisbaneLions_Sydney,84,799.0,1,42.0,36.055513,0.055499,0.178638,0,1,0,0,0,1,0.777213,0.734552
2,202101_BrisbaneLions_Sydney,158,1404.0,1,61.0,38.013156,1.107149,0.075831,0,0,0,1,0,0,0.289451,0.347986
3,202101_BrisbaneLions_Sydney,168,1480.0,0,52.0,40.459857,0.872894,0.102289,0,1,0,0,0,0,0.449251,0.450256
4,202101_BrisbaneLions_Sydney,224,1890.0,1,41.0,59.033889,0.893410,0.068149,0,0,0,1,0,0,0.171158,0.275946


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")